In [1]:
WINDOW_SIZE = 0.64 # km
SUBSET_SIZE = 1

In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib
matplotlib.rc('figure', figsize=(10, 10))

import geopandas as gpd
from shapely.geometry import Point
import shapely.geometry
import contextily as ctx
from shapely import wkt
from shapely import Polygon
import mercantile
import osmnx as ox
import numpy as np

import pyproj
from pyproj import Transformer
import rasterio


import dask.dataframe as dd
import math

import time

import logging
from tqdm import tqdm
logger = logging.getLogger()
logger.setLevel(logging.INFO)

ModuleNotFoundError: No module named 'geopandas'

In [3]:
def utm_to_latlon(easting, northing, zone_number=10, zone_letter="N"):
    """
    Convert UTM coordinates to Latitude and Longitude.

    Parameters:
        easting (float): The easting coordinate (meters).
        northing (float): The northing coordinate (meters).
        zone_number (int): The UTM zone number.
        zone_letter (str): The UTM zone letter.

    Returns:
        tuple: (latitude, longitude)
    """
    """
    utm_proj = pyproj.Proj(
        proj='utm',
        zone=zone_number,
        ellps='WGS84',
        south=(zone_letter.upper() < 'N')
    )
    latlon_proj = pyproj.Proj(proj='latlong', datum='WGS84')
    """
    
    transformer = Transformer.from_crs("EPSG:32610", "EPSG:4326") # no. 1 is CRS for UTM N 10 - CHANGE, no. 2 is std
    #longitude, latitude = pyproj.transform(utm_proj, latlon_proj, easting, northing)
    return transformer.transform(easting, northing)
    
    #return latitude, longitude

def extract_subgrids_with_bounds_utm(raster_filepath, origin, pixel_size_m=10, subgrid_size=64, null_value=-3.4e+38, tol=1e-6, zone_number=33, zone_letter='T'):
    with rasterio.open(raster_filepath) as src:
        raster = src.read(1)
    
    subgrids_with_bounds = []
    rows, cols = raster.shape
    
    for i in range(0, rows, subgrid_size):
        for j in range(0, cols, subgrid_size):
            subgrid = raster[i:i+subgrid_size, j:j+subgrid_size]
            
            contains_null = np.any(np.isclose(subgrid, null_value, atol=tol))
            
            if not contains_null:
                # Calculate bounds in UTM
                min_easting = origin[0] + j * pixel_size_m
                max_easting = min_easting + subgrid_size * pixel_size_m
                max_northing = origin[1] - i * pixel_size_m  # Assuming origin is top-left
                min_northing = max_northing - subgrid_size * pixel_size_m
                
                # Convert UTM bounds to lat/lon for polygon creation
                min_lat, min_lon = utm_to_latlon(min_easting, min_northing, zone_number, zone_letter)
                max_lat, max_lon = utm_to_latlon(max_easting, max_northing, zone_number, zone_letter)
                
                # Create a polygon
                polygon = Polygon([
                    (min_lon, min_lat),
                    (min_lon, max_lat),
                    (max_lon, max_lat),
                    (max_lon, min_lat)
                ])
                
                subgrids_with_bounds.append((subgrid, polygon))
    
    return subgrids_with_bounds


In [4]:
def bound_dots (poly): return [Point(vtx) for vtx in list(coord_window(34.0522,-118.2437).exterior.coords)]

In [5]:
def map_plot(coords, clr="red", ax=None):
    """plot coords on a map"""

    try:
        gdf = gpd.GeoDataFrame(geometry=coords, crs="EPSG:4326").to_crs(epsg=3857)
    except:
        gdf = gpd.GeoDataFrame(geometry=coords['geometry'], crs="EPSG:4326").to_crs(epsg=3857)
    if ax is None:
        fig, ax = plt.subplots(figsize=(30, 30))
    gdf.plot(ax=ax, facecolor='none', edgecolor=clr)
    ax.patch.set_edgecolor(clr)
    ax.patch.set_linewidth(1)
    ctx.add_basemap(ax, zoom="auto", source=ctx.providers.OpenStreetMap.Mapnik)

In [6]:
def annotate_gs (ax,gs,text,round_=False,grid=False,fontsize=10):
    if grid:
        geoms = gpd.GeoSeries(gs.flatten())
        geoms.crs = "EPSG:4326"
        geoms = geoms.to_crs(epsg=3857)

    else:
        geoms = gs
    for i, geom in enumerate(geoms):
        x, y = geom.centroid.x, geom.centroid.y
        t = str(text[i]).split(".")[0] if round_ else text[i]
        ax.annotate(t, xy=(x, y), ha='center', va='center', color='red',fontsize=fontsize)

In [7]:
def multi_map_plot(colors, *args):
    """plot multiple points/polygons etc on map"""
    fig, ax = plt.subplots()

    if len(colors) < len(args): colors *= math.ceil(len(args)/len(colors))
    for i, arg in enumerate(tqdm(args)): 
        map_plot(arg, ax=ax, clr=colors[i])

    ax.patch.set_edgecolor('yellow')  
    ax.patch.set_linewidth(1)  
    ctx.add_basemap(ax, zoom="auto", source=ctx.providers.OpenStreetMap.Mapnik)
    
    return ax


In [8]:
def sindex_search (aoi, to_search):
    sindex = to_search['geometry'].sindex
    bbox = aoi.bounds
    possible_matches_index = list(sindex.intersection(bbox))
    possible_matches = to_search.iloc[possible_matches_index]
    possible_matches.set_geometry("geometry")
    precise_matches = possible_matches[possible_matches['geometry'].intersects(aoi)]
    return precise_matches

In [9]:
ftp_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
def get_footprints (quad_keys,aoi_shape):
    """return all building footprint data in aoi"""
    ret = gpd.GeoDataFrame([])
    for qkey in quad_keys: # region is small but incase stn is between 2 qkeys
        # findn url
        rows = ftp_links[ftp_links['QuadKey'] == qkey]
        if rows.empty: return rows
        url = rows.sort_values(by="Size").iloc[0,:]["Url"] # choose url with highest info content        

        df2 = pd.read_json(url,lines=True)
        df2["geometry"] = df2["geometry"].apply(shapely.geometry.shape)

        gdf = gpd.GeoDataFrame(df2, crs=4326)  
            
        # use spatial index to find building footprints in our area of interest.
        sindex = gdf.sindex
        bbox = aoi_shape.bounds
        possible_matches_index = list(sindex.intersection(bbox))
        possible_matches = gdf.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(aoi_shape)]
        # sort out buildings from final matches that don't have height
        keep = precise_matches.apply(lambda row: row["properties"] != {"height": -1.0}, axis=1)
        ret = ret.append(precise_matches[keep])
    return ret
#footprints = get_footprints([23012311], coord_window(34.0522,-118.2437))

In [10]:
def get_osm_features (poly):
    PLT = False
    tmp = poly.bounds
    NSEW = (tmp[3], tmp[1], tmp[2], tmp[0])
    
    def get_tags (tags): 
        try:
            gdf = ox.features_from_bbox(*NSEW,tags=tags)
            key = list(tags.keys())[0]
            gdf = gdf.loc["way",:][[key, "geometry"]]
            gdf = gdf.rename(columns={key: "feature"})
            return gdf
        except Exception as e:
            return None
    
    # get infastrucutre
    ## roads
    #G = ox.graph_from_bbox(*NSEW,network_type="drive_service")
    #gdf_roads = ox.graph_to_gdfs(G, nodes=False)
    #buffer_size = 0.00005  # This is in degrees, adjust as needed
    #gdf_roads['geometry'] = gdf_roads['geometry'].buffer(buffer_size)
    
    
    ## landuse
    landuse_foi = ["residential", "industrial", "commercial", "greenfield", "brownfield", "grass", "forest", "farm", 
                  "orchard", "vineyard"]
    gdf_landuse = get_tags({"landuse": landuse_foi})

    # water
    water_foi = ["river", "canal", "stream", "drain"]
    gdf_water = get_tags({"waterway": water_foi})
    
    # get natural features of interest
    nat_foi = ['water', 'wood', 'grassland', 'heath', 'scrub', 'wetland', 'bare_rock']
    gdf_nat = get_tags({'natural': nat_foi})
    
    # get highways
    #road_foi = ["motorway", "trunk", "primary", "secondary", "tertiary", "residential"]
    road_foi = ["road"]
    gdf_road = get_tags({"highway": road_foi})

    #map_plot(gdf_buildings['geometry'],clr="green")
    if PLT:
        multi_map_plot(
            ['red',"blue","green","black"],
            bound_dots(coord_window(34.0522,-118.2437)),
            gdf_nat,
            gdf_parks,
            gdf_landuse
        )
    return {"landuse": gdf_landuse, "water": gdf_water, "natural": gdf_nat, "highway": gdf_road},{"landuse": landuse_foi, "water": water_foi, "natural": nat_foi, "highway": road_foi}

In [11]:
def get_bbox_from_tl (fn,easting,northing):
    with rasterio.open(fn) as src:
        shp = src.read(1).shape

    tl = (easting, northing)
    br = (easting+shp[1]*10,northing-shp[0]*10)

    top_left_lat, top_left_lon = utm_to_latlon(*tl)
    bottom_right_lat, bottom_right_lon = utm_to_latlon(*br)
    
    # Create Polygon
    polygon = Polygon([
        (top_left_lon, top_left_lat),  # Top-left
        (top_left_lon, bottom_right_lat),  # Top-right
        (bottom_right_lon, bottom_right_lat),  # Bottom-right
        (bottom_right_lon, top_left_lat)  # Bottom-left
    ])
    
    return polygon

In [12]:
filepath = './heatmaps/sf/rasters_d1/am_t_f.tif'
easting, northing = 542685,4187515

####### file dependent stuff

e = extract_subgrids_with_bounds_utm(filepath,(easting,northing))
df = pd.DataFrame(e, columns=['Subgrid', 'Bounds'])
grid_bbox = get_bbox_from_tl(filepath,easting,northing)

# add col to df
df["height_grid"] = np.empty(df.shape[0],dtype=object)
df["num_ftrs"] = np.empty(df.shape[0],dtype=object)
df["num_heights"] = np.empty(df.shape[0],dtype=object)

# get quadkeys that are in our aoi
minx, miny, maxx, maxy = grid_bbox.bounds
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(int(mercantile.quadkey(tile)))
quad_keys = list(quad_keys)

to_drop = []

############# SUBGRID DEPENDENT STUFF
for curr_subgrid in tqdm(list(df.index)):
    vertices_poly = df.iloc[curr_subgrid,:]['Bounds'] # subgrid Polygon
    minx, miny, maxx, maxy = vertices_poly.bounds # reset bounds to the current subgrid

    # query ms inside that window to check if there is building data (WITH HEIGHT) 
    footprints = get_footprints(quad_keys,vertices_poly)
    # not avail: drop,next.
    if footprints.empty: to_drop.append(curr_subgrid)

    ##### GET FEATURES BASE
    features, feature_tags = get_osm_features(vertices_poly)

    ##### CREATE GRID WITH BOUNDS
    res = 64
    grid = np.empty((res,res),dtype=object)
    # make res x res amount of boxes (+1 for edge)
    bounds_x = np.linspace(minx,maxx,num=res+1)
    bounds_y = np.linspace(maxy,miny,num=res+1) #### CHANGE MAXy-MINy

    for x_i, xleft in enumerate(bounds_x[:-1]):
        xright = bounds_x[x_i+1]
        for y_i, yup in enumerate(bounds_y[:-1]):
            ydown = bounds_y[y_i+1]
            poly = shapely.geometry.box(xleft,ydown,xright,yup)
            grid[y_i][x_i] = poly


    # for each poly in grid, get height of building (or 0 for street)
    heights_grid = np.zeros((res,res))
    feature_grids = {}
    feature_arrs = {}
    num_heights = 0
    num_ftrs = {}

    for ftype in features.keys():
        tag_key = feature_tags[ftype]
        feature_grids[ftype] = np.array([[np.zeros(len(tag_key)) for _ in range(res)] for _ in range(res)])
        feature_arrs[ftype] = ["none"] * res
        num_ftrs[ftype] = 0


    for y,row in enumerate(grid):
        for x, square in enumerate(row):
            ##### RASTERIZE BUILDING HEIGHTS
            precise_matches = sindex_search (square,footprints)
            if precise_matches.shape[0] == 0: 
                heights_grid[y][x] = 0
            else:
                heights_grid[y][x] = max(pd.json_normalize(precise_matches['properties'])['height'])
                num_heights += 1

            ##### RASTERIZE FEATURES
            for ftype in features.keys():
                tag_key = feature_tags[ftype]
                ftr_df = features[ftype]
                try:
                    precise_matches = sindex_search (square,ftr_df)
                except: 
                    continue
                onehot = np.zeros(len(tag_key))
                if precise_matches.shape[0] == 0:
                    feature_grids[ftype][y][x] = onehot
                    feature_arrs[ftype].append("none")
                    continue

                ftr = precise_matches.iloc[0,:]['feature'] # residential, commercial, river...
                onehot[tag_key.index(ftr)] = 1
                feature_grids[ftype][y][x] = onehot
                feature_arrs[ftype].append(ftr)
                num_ftrs[ftype] += 1
    df.loc[curr_subgrid,"feature_grids"] = [feature_grids]
    df.at[curr_subgrid,"height_grid"] = heights_grid
    df.at[curr_subgrid,"num_ftrs"] = num_ftrs
    df.at[curr_subgrid,"num_heights"] = num_heights

df = df.drop(pd.Index(to_drop))


NameError: name 'rasterio' is not defined

In [13]:
df.to_pickle("./sf_am.pkl")

NameError: name 'df' is not defined

In [15]:
am, af, pm = pd.read_pickle('./sf_am.pkl'), pd.read_pickle('./sf.pkl'), pd.read_pickle('./sf_pm.pkl')

FileNotFoundError: [Errno 2] No such file or directory: './sf_am.pkl'

In [16]:
am['AMafPM'] = [[1, 0, 0] for i in range(am.shape[0])];af['AMafPM'] = [[0, 1, 0] for i in range(af.shape[0])];pm['AMafPM'] = [[0, 0, 1] for i in range(pm.shape[0])]

NameError: name 'am' is not defined

In [17]:
final_df = pm.append(am.append(af,ignore_index=True),ignore_index=True)

NameError: name 'pm' is not defined

In [18]:
final_df.to_pickle("./final.pkl")

NameError: name 'final_df' is not defined

In [14]:
### TESTING


In [19]:
row = df.iloc[0,:]
res=64
minx, miny, maxx, maxy = row['Bounds'].bounds
grid = np.empty((res,res),dtype=object)
# make res x res amount of boxes (+1 for edge)
bounds_x = np.linspace(minx,maxx,num=res+1)
bounds_y = np.linspace(maxy,miny,num=res+1)

for x_i, xleft in enumerate(bounds_x[:-1]):
    xright = bounds_x[x_i+1]
    for y_i, yup in enumerate(bounds_y[:-1]):
        ydown = bounds_y[y_i+1]
        poly = shapely.geometry.box(xleft,ydown,xright,yup)
        grid[y_i][x_i] = poly

ax = multi_map_plot(['black', 'red', 'blue'],gpd.GeoSeries(row['Bounds']),*grid)
#annotate_gs(ax,grid,pool(64,row['height_grid']).flatten(),round_=True,grid=True,fontsize=3)
annotate_gs(ax,grid,row['height_grid'].flatten(),round_=True,grid=True,fontsize=4)

plt.show()


NameError: name 'df' is not defined